In [20]:
%reload_kedro

[02/08/24 01:24:39] INFO     Resolved project path as: /scratch/rahit/modspy-data.                  ]8;id=672918;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=252431;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Resolved project path as: /scratch/rahit/modspy-data.                  ]8;id=12178;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=778501;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/08/24 01:24:46] INFO     Kedro project ModSpy-Data                                              ]8;id=713822;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=727984;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Kedro project ModSpy-Data                                              ]8;id=930068;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=814161;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=145773;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=890472;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=246366;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=918144;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

In [21]:
import itertools
from collections import defaultdict
from rich import print
from IPython.display import display, HTML

import dask
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, progress, performance_report
from dask_jobqueue import SLURMCluster

In [22]:
cluster = SLURMCluster(
    cores=16,  # Number of cores per job
    processes=1,
    memory='32GB',  # Memory allocated to each worker
    walltime='00:30:00',  # Walltime limit for each job
    # Specify any additional SLURM or Dask configurations as needed
)

[02/08/24 01:24:47] WARNING  /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/node.p ]8;id=49712;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=902306;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/warnings.py#109\109]8;;\
                             y:182: UserWarning: Port 8787 is already in use.                                      
                             Perhaps you already have a cluster running?                                           
                             Hosting the HTTP server on port 34342 instead                                         
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     State start                                                          ]8;id=342018;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=647013;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#1615\1615]8;;\

                    INFO       Scheduler at:  tcp://10.29.84.182:36718                            ]8;id=892883;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=336282;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3860\3860]8;;\

                    INFO       dashboard at:                    :34342                            ]8;id=672894;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=86146;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3862\3862]8;;\

In [23]:
client = Client(cluster)  # start distributed scheduler locally.

[02/08/24 01:24:48] INFO     Receive client connection:                                           ]8;id=912296;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=623157;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5228\5228]8;;\
                             Client-c266267c-c64a-11ee-8fd0-707990595d1c                                           

                    INFO     Starting established connection to tcp://10.29.84.182:41028                ]8;id=823737;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=264190;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#867\867]8;;\

In [24]:
# Scale the cluster to the desired number of workers
cluster.scale(jobs=1)  # Request 10 jobs, adjust based on your needs

In [25]:
!pwd

/scratch/rahit/modspy-data/notebooks/monarch_gnn


In [26]:
nodes_df = dd.read_csv('../../data/01_raw/monarch/monarch-kg_nodes.tsv', sep='\t', usecols=['id', 'category', 'name', 'in_taxon', 'in_taxon_label', 'symbol'], dtype={'id':'object', 
                        'category':'object', 'name':'object', 'in_taxon':'object', 'in_taxon_label':'object', 'symbol':'object'})
edges_df = dd.read_csv('../../data/01_raw/monarch/monarch-kg_edges.tsv', sep='\t', usecols=['id', 'original_subject', 'predicate', 'original_object', 'category', 'subject', 'object'],
                      dtype={'id':'object', 'original_subject':'object', 'predicate':'object', 'original_object':'object', 'category':'object', 'subject':'object', 'object':'object'})

[02/08/24 01:25:05] INFO     Register worker <WorkerState 'tcp://10.29.83.200:39622', name:       ]8;id=18255;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=975121;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4212\4212]8;;\
                             SLURMCluster-0, status: init, memory: 0, processing: 0>                               

                    INFO     Starting worker compute stream, tcp://10.29.83.200:39622             ]8;id=395044;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=791322;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5468\5468]8;;\

                    INFO     Starting established connection to tcp://10.29.83.200:60878                ]8;id=231703;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=733597;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#867\867]8;;\

In [8]:
edges_df.head()

[02/07/24 19:13:27] INFO     Register worker <WorkerState 'tcp://10.29.83.188:43549', name:       ]8;id=934367;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=492190;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4212\4212]8;;\
                             SLURMCluster-0, status: init, memory: 0, processing: 0>                               

                    INFO     Starting worker compute stream, tcp://10.29.83.188:43549             ]8;id=412962;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=91683;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5468\5468]8;;\

                    INFO     Starting established connection to tcp://10.29.83.188:38936                ]8;id=737969;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=909091;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#867\867]8;;\

,id,original_subject,predicate,original_object,category,subject,object
0,uuid:68d6e706-9bb0-11ee-b780-6b2918cfaf31,NCBIGene:64170,biolink:causes,OMIM:212050,biolink:CausalGeneToDiseaseAssociation,HGNC:16391,MONDO:0008905
1,uuid:68d6e707-9bb0-11ee-b780-6b2918cfaf31,NCBIGene:51256,biolink:causes,OMIM:248000,biolink:CausalGeneToDiseaseAssociation,HGNC:21066,MONDO:0009544
2,uuid:68d6e708-9bb0-11ee-b780-6b2918cfaf31,NCBIGene:28981,biolink:causes,OMIM:617895,biolink:CausalGeneToDiseaseAssociation,HGNC:14313,MONDO:0033485
3,uuid:68d6e709-9bb0-11ee-b780-6b2918cfaf31,NCBIGene:8216,biolink:causes,OMIM:616564,biolink:CausalGeneToDiseaseAssociation,HGNC:6742,MONDO:0014693
4,uuid:68d6e70a-9bb0-11ee-b780-6b2918cfaf31,NCBIGene:6505,biolink:contributes_to,OMIM:615232,biolink:CorrelatedGeneToDiseaseAssociation,HGNC:10939,MONDO:0014092


Adding nodes' category information by merging node dataframe on the subject and object column.

In [27]:
nodes_df = nodes_df.set_index('id')

In [10]:
_edf = edges_df.merge(nodes_df, left_on='subject', right_index=True, suffixes=('_ndf', '_edf'))    # @TODO swap _ndf and _edf 
_edf = _edf.rename(columns={'category_ndf': 'subject_category', 'category_edf': 'edge_category'})
print(f"Columns after merging on subject category: {_edf.columns}")

_edf = _edf.merge(nodes_df, left_on='object', right_index=True, suffixes=('_ndf', '_edf'))
_edf = _edf.rename(columns={'category': 'object_category'})
print(f"Columns after merging on object category: {_edf.columns}")

Columns after merging on subject category: Index(['id', 'original_subject', 'predicate', 'original_object',
       'subject_category', 'subject', 'object', 'edge_category', 'name',
       'in_taxon', 'in_taxon_label', 'symbol'],
      dtype='object')

Columns after merging on object category: Index(['id', 'original_subject', 'predicate', 'original_object',
       'subject_category', 'subject', 'object', 'edge_category', 'name_ndf',
       'in_taxon_ndf', 'in_taxon_label_ndf', 'symbol_ndf', 'object_category',
       'name_edf', 'in_taxon_edf', 'in_taxon_label_edf', 'symbol_edf'],
      dtype='object')

In [11]:
edges = _edf[['id','subject', 'subject_category', 'predicate', 'edge_category', 'object_category', 'object']]
print(f"Columns after renaming and triming: {edges.columns}")

Columns after renaming and triming: Index(['id', 'subject', 'subject_category', 'predicate', 'edge_category',
       'object_category', 'object'],
      dtype='object')

In [12]:
# edges['edge_key']  = edges.apply(lambda row:(row['subject_category'], row['predicate'], row['object_category']), axis=1, meta=pd.Series(dtype=(set<str>)))
edges = edges.reset_index(drop=True)
edges.columns

Index(['id', 'subject', 'subject_category', 'predicate', 'edge_category',
       'object_category', 'object'],
      dtype='object')

In [13]:
edges.to_parquet('../../data/02_intermediate/monarch/edges_with_node_cat', write_index=False)  

In [19]:
!ls -ahl '../../data/02_intermediate/monarch/edges_with_node_cat'

total 168M
drwxr-x--- 2 rahit rahit  33K Feb  7 19:15 .
drwxr-x--- 3 rahit rahit  33K Feb  7 18:17 ..
-rw-r----- 1 rahit rahit 100M Feb  7 19:15 part.0.parquet
-rw-r----- 1 rahit rahit  16M Feb  7 19:15 part.1.parquet
-rw-r----- 1 rahit rahit  53M Feb  7 19:15 part.2.parquet


In [15]:
edges.shape[0].compute()

11412471

In [34]:
cluster.close()

[02/08/24 01:50:47] INFO     Scheduler closing...                                                 ]8;id=426356;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=993055;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3925\3925]8;;\

                    INFO     Scheduler closing all comms                                          ]8;id=32209;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=256358;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3947\3947]8;;\

                    INFO     Connection to tcp://10.29.83.200:60878 has been closed.                    ]8;id=476174;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=766114;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.83.200:39622', name:         ]8;id=744247;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=469053;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-0, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.83.200:39622                                ]8;id=722640;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=231679;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Lost all workers                                                     ]8;id=667104;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=441229;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4875\4875]8;;\

In [35]:
client.close()